# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# План проекта

    1. Загрузить и подготовить данные. Пояснить порядок действий.
    
    2. Обучить и проверить модель для каждого региона:
    2.1. Разбить данные на обучающую и валидационную выборки в соотношении 75:25.
    2.2. Обучить модель и сделать предсказания на валидационной выборке.
    2.3. Сохранить предсказания и правильные ответы на валидационной выборке.
    2.4. Напечатать на экране средний запас предсказанного сырья и RMSE модели.
    2.5. Проанализировать результаты.
    
    3. Подготовиться к расчёту прибыли:
    3.1. Все ключевые значения для расчётов сохранить в отдельных переменных.
    3.2. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе.
    3.3. Написать выводы по этапу подготовки расчёта прибыли.
    
    4. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
    4.1. Выбрать скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины.
    4.2. Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
    4.3. Рассчитать прибыль для полученного объёма сырья.
    
    5. Посчитать риски и прибыль для каждого региона:
    5.1. Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    5.2. Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    5.3. Написать выводы: предложить регион для разработки скважин и обосновать выбор.

# Описание данных

Данные геологоразведки трёх регионов находятся в файлах:

    geo_data_0.csv 
    geo_data_1.csv
    geo_data_2.csv 
    id — уникальный идентификатор скважины;
    f0, f1, f2 — три признака точек;
    product — объём запасов в скважине (тыс. баррелей).

# Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 1. Загрузка и подготовка данных

Загрузим все необходимые библиотеки и все три датасета, присвоив им имена.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from IPython.display import display

In [2]:
gd0 = pd.read_csv('/datasets/geo_data_0.csv')
gd1 = pd.read_csv('/datasets/geo_data_1.csv')
gd2 = pd.read_csv('/datasets/geo_data_2.csv')

Рассмотрим все три по очереди.

In [3]:
gd0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
gd0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
gd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
gd1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
gd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
gd2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


По 100000 скважин в каждом регионе, пропусков нет, но для наших целей столбец id абсолютно нерелвантен и будет сбивать. Удалим его из всех трех датасетов.

In [9]:
gd0 = gd0.drop(['id'], axis=1)
gd1 = gd1.drop(['id'], axis=1)
gd2 = gd2.drop(['id'], axis=1)

Преобразование признаков не требуется. Данные готовы к анализу.

# 2. Обучение и проверка модели

Сначала разобьем данные каждого датасета на обучающую и валидационную выборки в пропорции 75:25.

In [10]:
gd0_train, gd0_valid = train_test_split(gd0, test_size=0.25, random_state=12345)
gd1_train, gd1_valid = train_test_split(gd1, test_size=0.25, random_state=12345)
gd2_train, gd2_valid = train_test_split(gd2, test_size=0.25, random_state=12345)

Проверим себя

In [11]:
gd0_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 27212 to 86498
Data columns (total 4 columns):
f0         75000 non-null float64
f1         75000 non-null float64
f2         75000 non-null float64
product    75000 non-null float64
dtypes: float64(4)
memory usage: 2.9 MB


In [12]:
gd0_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 71751 to 43558
Data columns (total 4 columns):
f0         25000 non-null float64
f1         25000 non-null float64
f2         25000 non-null float64
product    25000 non-null float64
dtypes: float64(4)
memory usage: 976.6 KB


In [13]:
gd1_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 27212 to 86498
Data columns (total 4 columns):
f0         75000 non-null float64
f1         75000 non-null float64
f2         75000 non-null float64
product    75000 non-null float64
dtypes: float64(4)
memory usage: 2.9 MB


In [14]:
gd1_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 71751 to 43558
Data columns (total 4 columns):
f0         25000 non-null float64
f1         25000 non-null float64
f2         25000 non-null float64
product    25000 non-null float64
dtypes: float64(4)
memory usage: 976.6 KB


In [15]:
gd2_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 27212 to 86498
Data columns (total 4 columns):
f0         75000 non-null float64
f1         75000 non-null float64
f2         75000 non-null float64
product    75000 non-null float64
dtypes: float64(4)
memory usage: 2.9 MB


In [16]:
gd2_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 71751 to 43558
Data columns (total 4 columns):
f0         25000 non-null float64
f1         25000 non-null float64
f2         25000 non-null float64
product    25000 non-null float64
dtypes: float64(4)
memory usage: 976.6 KB


Все верно. Теперь приступим к обучению модели линейной регрессии. Для этого надо вычленить признаки и целевой признак для обучающей и валидационной выборок. Целевой признак - объем баррелей в скважине. Признаки - остальные.

In [17]:
features_gd0 = gd0_train.drop(['product'], axis=1)
target_gd0 = gd0_train['product']
features_gd1 = gd1_train.drop(['product'], axis=1)
target_gd1 = gd1_train['product']
features_gd2 = gd2_train.drop(['product'], axis=1)
target_gd2 = gd2_train['product']
features_valid_gd0 = gd0_valid.drop(['product'], axis=1)
target_valid_gd0 = gd0_valid['product'].reset_index(drop=True)  #индексация таргета
features_valid_gd1 = gd1_valid.drop(['product'], axis=1)
target_valid_gd1 = gd1_valid['product'].reset_index(drop=True)
features_valid_gd2 = gd2_valid.drop(['product'], axis=1)
target_valid_gd2 = gd2_valid['product'].reset_index(drop=True)
target_gd0

27212    147.370612
7866     147.630053
62041     77.696728
70185     55.210501
82230    113.891723
            ...    
4094     124.380793
85412    144.874913
2177     134.967255
77285     64.494357
86498    151.514894
Name: product, Length: 75000, dtype: float64

Обучим модели.

In [18]:
model_gd0 = LinearRegression().fit(features_gd0, target_gd0)
model_gd1 = LinearRegression().fit(features_gd1, target_gd1)
model_gd2 = LinearRegression().fit(features_gd2, target_gd2)

Сделаем предсказания на валидационной выборке и преобразуем их в Series.

In [19]:
predictions_gd0 = model_gd0.predict(features_valid_gd0)
predictions_gd1 = model_gd1.predict(features_valid_gd1)
predictions_gd2 = model_gd2.predict(features_valid_gd2)
predictions_gd0

array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229])

Посчитаем средний запас предсказанного сырья и rmse модели в каждом регионе.

In [20]:
mse_gd0 = mean_squared_error(target_valid_gd0, predictions_gd0)
rmse_gd0 = mse_gd0 ** 0.5
print("Средний запас предсказанного сырья в первом регионе, тыс. барр.:", predictions_gd0.mean())
print("RMSE первый регион, тыс. барр.:", rmse_gd0)
print()
mse_gd1 = mean_squared_error(target_valid_gd1, predictions_gd1)
rmse_gd1 = mse_gd1 ** 0.5
print("Средний запас предсказанного сырья во втором регионе, тыс. барр.:", predictions_gd1.mean())
print("RMSE второй регион, тыс. барр.:", rmse_gd1)
print()
mse_gd2 = mean_squared_error(target_valid_gd2, predictions_gd2)
rmse_gd2 = mse_gd2 ** 0.5
print("Средний запас предсказанного сырья в третьем регионе, тыс. барр.:", predictions_gd2.mean())
print("RMSE третий регион, тыс. барр.:", rmse_gd2)

Средний запас предсказанного сырья в первом регионе, тыс. барр.: 92.59256778438038
RMSE первый регион, тыс. барр.: 37.5794217150813

Средний запас предсказанного сырья во втором регионе, тыс. барр.: 68.728546895446
RMSE второй регион, тыс. барр.: 0.893099286775616

Средний запас предсказанного сырья в третьем регионе, тыс. барр.: 94.96504596800489
RMSE третий регион, тыс. барр.: 40.02970873393434


## Вывод

Средний запас выше в первом и третьем регионах (92 и 94 тысячи баррелей соответственно), но лучше всех именно модель себя показала во втором регионе - 68 тысяч баррелей, но среднеквадратичная ошбика всего 0.89, тогда как в первом и третьем ошибки имели значение в 37 и 40 соответственно. Модель второго региона существенно снизила вероятность ошибки, а следовательно и возможных убытков. Второй регион, с точки зрения бизнеса, пока лидирует, на мой взгляд.

# 3. Подготовка к расчёту прибыли

Получив рабочие модели, нам необходимо подготовиться к расчету прибыли. Для этого необходимо зафиксировать в переменных все требуемые значения.

Бюджет на разработку скважин в регионе - 10 млрд рублей. Так как из всех скважин мы будем случайным образом выбирать  для анализа 500, а разрабатывать только 200 из них, то бюджет на разработку одной скважины составит 50 миллионов рублей. Зафиксируем бюджет в переменной budget, а количество скважин для анализа - в переменной sample.

In [21]:
budget = 10e9
sample = 500

Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. Зафиксируем это в переменной income.

In [22]:
income = 450000

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них будем выбирать регион с наибольшей средней прибылью. Порог в 2.5% зафиксируем в переменной limit.

In [23]:
limit = 0.025

Также нам необходимо рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Зафиксируем этот показатель в переменной min_vol. Для этого необходимо разделить budget на income.

In [24]:
min_vol = budget / income/200
min_vol

111.11111111111111

## Вывод

Итак, мы получили минимальный порог объема скважины в 111 тысяч баррелей нефти. Это больше, чем среднее прогнозируемое значение в любом из изучаемых нами регионов. Необходимо тщательнее подойти к подбору региона.

# 4. Расчёт прибыли и рисков 

В первую очередь для поиска наиболее подходящего региона нам потребуется написать формулу для подсчета прогнозируемой прибыли с каждого региона. В ней и будем производить сортировку 200 луших скважин.

In [25]:
def revenue(target,predictions,count):
    predictions = pd.Series(predictions)
    predictions = predictions.sort_values(ascending=False)
    top_200 = target[predictions.index][:200]
    product = top_200.sum()
    profit = product * income
    revenue_gd = profit - budget
    return revenue_gd

Теперь нам необходимо проверить нашу модель - посчитать необходимые показатели на реальных объемах скважин для всех трех регионов и на основе этих данных определить какой регион набиолее предпочтителен.

Первый регион

In [26]:
state = np.random.RandomState(12345)
 
values = []
for i in range(1000):
    target_subsample_1 = target_valid_gd0.sample(n=sample, replace=True, random_state=state)
    probs_subsample_1 = predictions_gd0[target_subsample_1.index]
    values.append(revenue(pd.Series(target_subsample_1).reset_index(drop=True), pd.Series(probs_subsample_1), 200))   
    

values = pd.Series(values)
 
risk = (values < 0).mean()
mean = values.mean()
confidence_interval = values.quantile(0.025), values.quantile(0.975)
print('Показатели для первого региона')
print()
print('Прогнозируемая прибыль с первого региона:', revenue(target_valid_gd0, predictions_gd0, 200))
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для первого региона

Прогнозируемая прибыль с первого региона: 3320826043.1398525
Средняя прибыль: 396164984.8023711
95%-й доверительный интервал: (-111215545.89049526, 909766941.5534225)
Риск убытков: 0.069


Второй регион

In [27]:
state = np.random.RandomState(12345)
 
values = []
for i in range(1000):
    target_subsample_1 = target_valid_gd1.sample(n=sample, replace=True, random_state=state)
    probs_subsample_1 = predictions_gd1[target_subsample_1.index]
    values.append(revenue(pd.Series(target_subsample_1).reset_index(drop=True), pd.Series(probs_subsample_1), 200))


values = pd.Series(values)
 
risk = (values < 0).mean()
mean = values.mean()
confidence_interval = values.quantile(0.025), values.quantile(0.975)
print('Показатели для второго региона')
print()
print('Прогнозируемая прибыль со второго региона:', revenue(target_valid_gd1, predictions_gd1, 200))
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для второго региона

Прогнозируемая прибыль со второго региона: 2415086696.681511
Средняя прибыль: 456045105.7866609
95%-й доверительный интервал: (33820509.39898549, 852289453.866036)
Риск убытков: 0.015


Третий регион

In [28]:
state = np.random.RandomState(12345)
 
values = []
for i in range(1000):
    target_subsample_1 = target_valid_gd2.sample(n=sample, replace=True, random_state=state)
    probs_subsample_1 = predictions_gd2[target_subsample_1.index]
    values.append(revenue(pd.Series(target_subsample_1).reset_index(drop=True)  , pd.Series(probs_subsample_1), 200))
    

values = pd.Series(values)
 
risk = (values < 0).mean()
mean = values.mean()
confidence_interval = values.quantile(0.025), values.quantile(0.975)
print('Показатели для третьего региона')
print()
print('Прогнозируемая прибыль с третьего региона:', revenue(target_valid_gd2, predictions_gd2, 200))
print('Средняя прибыль:', mean)
print('95%-й доверительный интервал:', confidence_interval)
print('Риск убытков:', risk)

Показатели для третьего региона

Прогнозируемая прибыль с третьего региона: 2710349963.5998325
Средняя прибыль: 404403866.56835675
95%-й доверительный интервал: (-163350413.39559925, 950359574.9237995)
Риск убытков: 0.076


# Вывод

Проведя весь требуемый анализ мы можем однозначно сказать, что самый перспективный регион - второй. Показатели средней прибыли, прогнозируемой прибыли и границы доверительного интервала превосходят аналогичные в других регионах, а также риск убытков ниже всех, поэтому рекомендуем в первую очередь пристальнее изучить именно его. Именно изучить, а не сразу отправиться на разработку, ведь объем вложений предполагается крайне большой, а рисковать, опираясь только на моделирование мы не можем порекомендовать.

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован